In [1]:
import requests, json, sys
sys.path.insert(1, '../../scripts/')
from s3_support import *

In [2]:
URL = "https://benchmarksapi.qgiv.com/benchmarks/"
URL_TEST = "https://benchmarksapi.qgiv.com/test/"

# test

In [4]:
rsp = requests.get(URL_TEST)
rsp.content

b'{\n  "success": "1",\n  "message": "10 entries retreived"\n}\n'

In [7]:
d = {
    "key": 'tIHLM2vNlBwvlZlqdKy8',
    "filters": [
        {"org":444339}, 
        {"date_min": "2024-01-01"}, 
        {"date_max": "2024-12-31"}
    ], 
    "insight": "form_conversion" 
}

In [10]:
d = {
    'key': 'tIHLM2vNlBwvlZlqdKy8',
    'filters': [
        {'org': 441967},
        {'date_min': '2019-01-01'},
        {'date_max': '2021-01-01'}
    ],
    'insight': 'form_type'
}

In [8]:
rsp = requests.post(URL, json=d)
rsp.status_code

200

In [9]:
rsp.content

b'{\n  "success": "1",\n  "message": {\n    "sample_size": 148,\n    "data": {\n      "mean": 0.13426635820455415,\n      "median": 0.05635999451357653\n    }\n  }\n}\n'

In [13]:
d = {
    'key': 'tIHLM2vNlBwvlZlqdKy8',
    'comparison_org': 441967,
    'filters': [
        {'date_min': '2019-01-01'},
        {'date_max': '2021-01-01'}
    ],
    'insight': 'form_type'
}

In [14]:
rsp = requests.post(URL, json=d)
rsp.status_code

200

In [15]:
json.loads(rsp.content)

{'success': '1',
 'message': {'sample_size': 4158,
  'data': {'auction': {'mean': 2554.3120586820583, 'median': 0.0},
   'barnstorm': {'mean': 5.8534752284752285, 'median': 0.0},
   'p2p': {'mean': 417094.7077296777, 'median': 0.0},
   'qgiv': {'mean': 1783692.9216113514, 'median': 10904.4},
   'auction_percentage': {'mean': 0.03349818343722017, 'median': 0.0},
   'barnstorm_percentage': {'mean': 0.002167371993679333, 'median': 0.0},
   'p2p_percentage': {'mean': 0.12651807631341327, 'median': 0.0},
   'qgiv_percentage': {'mean': 0.8378163682556872, 'median': 1.0}}}}

In [16]:
json.loads(rsp.content)['message'].keys()

dict_keys(['sample_size', 'data'])

# query

In [253]:
d = {
    'key': 'tIHLM2vNlBwvlZlqdKy8',
    'filters': [
        {'date_min': '2019-01-01'},
        {'date_max': '2020-01-01'}
    ],
    'insight': 'churn'
}

In [254]:
rsp = requests.post(URL, json=d)
rsp.status_code

500

In [255]:
rsp.content

b'<!doctype html>\n<html lang=en>\n  <head>\n    <title>TypeError: Object of type int64 is not JSON serializable\n // Werkzeug Debugger</title>\n    <link rel="stylesheet" href="?__debugger__=yes&amp;cmd=resource&amp;f=style.css">\n    <link rel="shortcut icon"\n        href="?__debugger__=yes&amp;cmd=resource&amp;f=console.png">\n    <script src="?__debugger__=yes&amp;cmd=resource&amp;f=debugger.js"></script>\n    <script>\n      var CONSOLE_MODE = false,\n          EVALEX = true,\n          EVALEX_TRUSTED = false,\n          SECRET = "NlO8vDAVshZWCYyddH42";\n    </script>\n  </head>\n  <body style="background-color: #fff">\n    <div class="debugger">\n<h1>TypeError</h1>\n<div class="detail">\n  <p class="errormsg">TypeError: Object of type int64 is not JSON serializable\n</p>\n</div>\n<h2 class="traceback">Traceback <em>(most recent call last)</em></h2>\n<div class="traceback">\n  <h3></h3>\n  <ul><li><div class="frame" id="frame-140024210484976">\n  <h4>File <cite class="filename">"

In [ ]:
select
    count(distinct(do.org)) as sample_size,
    avg(do.volume) as average_lifetime_value
from public.donors_orgs as do 
    left join production.transactions as t on t.org=do.org

# scratch

In [187]:
q = '''select
            sum(distinct(case when month=12 then amount else 0 end)) as december_vol,
            sum(distinct(case when month!=12 then amount else 0 end)) as rest_vol
        from transactions
        where status='A'
        group by org, year'''
dec = redshift_query_read(q, schema='production')
dec['ratio'] = dec['december_vol'] / dec['rest_vol']

dec['ratio'].mean(), dec['ratio'].median()

(inf, 0.1346976453915463)

In [195]:
dec[dec['rest_vol']>1000].tail()

,december_vol,rest_vol,ratio
32226,0.0,1296.00,0.0
32229,525.0,1050.00,0.5
32234,0.0,1785.01,0.0
32236,0.0,2035.00,0.0
32271,0.0,1586.00,0.0


In [248]:
dec[dec['rest_vol']>2500]['ratio'].mean(), dec[dec['rest_vol']>2500]['ratio'].median()

(0.37887318907238665, 0.16025225977925472)

In [196]:
dec[dec['rest_vol']>1000]['ratio'].mean(), dec[dec['rest_vol']>1000]['ratio'].median()

(0.43818896919312006, 0.15854253820009961)

In [249]:
dec[dec['rest_vol']>100]['ratio'].mean(), dec[dec['rest_vol']>100]['ratio'].median()

(0.6942049656733187, 0.15003933725115554)

## support

In [137]:
def build_and_execute_query(q, where_clause=[], parameters=[], joins=[], group_on=None):
    if len(joins) > 0:
        q += ' left join ' + ' left join '.join(joins)

    if len(where_clause) > 0:
        q += ' where ' + ' and '.join(where_clause)
    
    if group_on is not None:
        q += ' group by {}'.format(group_on)
        
    print(q)
    print(parameters)

    df = redshift_query_read(q.format(*parameters), schema='production')

    return df

In [160]:
def get_benchmarks_data(_input):
    where_clause = []
    params = []
    joins = []
    data = {'sample_size': 0, 'data': None}

    filters = _input.get('filters', [])
    if len(filters) > 0:
        for k in filters:
            if k == 'state':
                where_clause.append("o.state='{}'")
                params.append(filters['state'])
                joins.append('production.organization as o on t.org=o.id')
            elif k == 'volume':
                # org size
                # what buckets will we use here?
                pass
            elif k == 'date_min':
                where_clause.append("t.date>='{}'")
                params.append(filters['date_min'])
            elif k == 'date_max':
                where_clause.append("t.date<='{}'")
                params.append(filters['date_max'])
            elif k == 'trailing':
                # trailing30, trailing180, trailing365, etc.
                where_clause.append("t.date>='{}'")
                params.append("current_day - interval '{} day'".foramt(filters['trailing']))
            elif k == 'ntee':
                where_clause.append("o.ntee LIKE '%{}%'")
                params.append(filters['ntee'])
                joins.append('production.organization as o on t.org=o.id')
            elif k == 'tag':
                where_clause.append("o.tags LIKE '%{}%'")
                params.append(filters['tags'])
                joins.append('production.organization as o on t.org=o.id')
            elif k == 'source':
                if _input['source'] == 'don_form':
                    where_clause.append("t.source='{}'")
                    params.append('don_form')
                elif _input['source'] == 'mobile':
                    where_clause.append("t.source='{}'")
                    params.append('mobile')
                elif _input['source'] == 'p2p':
                    where_clause.append("t.source='{}'")
                    params.append('p2p')
            elif k == 'formtype':
                joins.append('production.form as f on f.id=t.form')

                if _input['formtype'] == 'p2p':
                    where_clause.append("f.type={}")
                    params.append(3)
                elif _input['formtype'] == 'yearround':
                    # qgiv
                    where_clause.append("f.type={}")
                    params.append(1)
                elif _input['formtype'] == 'auction':
                    where_clause.append("f.type={}")
                    params.append(5)

    '''
    Insights:
    amount left to be raised, 
    at-risk donors, donor lifetime value, lapsed donors, new donors, 
    new vs repeat donors, retention rate, churn rate, 

    attendees bidding, attendees checked-in, auction constituents, 
    auction overview, average auction item sale price, average bid,
    average number of bids, bid activity over time, bids by increment amount,
    bids by source, higest bid, item performance, 

    contributions by associated info, contributions by date, 
    recent transactions, contributions by amount segment, contributions by form, 
    contributions by restriction, 
    contributions by text campaign,

    packages sold, contributions by event, days left to event, 
    p2p constituents, p2p overview, 
    '''
    if _input['insight'] == 'form_type':  # contributions by form type
        q = '''select 
                    t.org as org,
                    f.type as type,
                    sum(t.amount) as volume,
                    count(distinct(t.id)) as count
                from production.transactions as t'''
        joins.append("production.form as f on t.form=f.id")
        group_on = 'f.type, t.org'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)
        
        def interpret_form_type(form_type):
            if form_type == 1:
                return 'qgiv'
            elif form_type == 2:
                return 'barnstorm'
            elif form_type == 3:
                return 'p2p'
            elif form_type == 5:
                return 'auction'
        
        df['type'] = df['type'].apply(interpret_form_type)
        df = df[~df['type'].isna()]
        df_pivot = df.pivot(index='org', columns='type', values='volume').reset_index().fillna(0)

        type_cols = [c for c in df_pivot.columns if c!='org']
        for c in df_pivot.columns:
            if c != 'org':
                df_pivot['{}_percentage'.format(c)] = df_pivot[c] / df_pivot[type_cols].sum(axis=1)

        data = {
            'sample_size': len(df_pivot),
            'data': df_pivot[[c for c in df_pivot.columns if c!='org']].agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] == 'payment_method':  # contributions by payment method
        q = '''select 
                    t.org as org,
                    t.payment_type as payment_type,
                    sum(t.amount) as volume,
                    count(distinct(t.id)) as count
                from production.transactions as t'''
        group_on = 't.org, t.payment_type'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)
        df = df[~df['payment_type'].isna()]
        df_pivot = df.pivot(index='org', columns='payment_type', values='volume').reset_index().fillna(0)

        type_cols = [c for c in df_pivot.columns if c!='org']
        for c in df_pivot.columns:
            if c != 'org':
                df_pivot['{}_percentage'.format(c)] = df_pivot[c] / df_pivot[type_cols].sum(axis=1)

        data = {
            'sample_size': len(df_pivot),
            'data': df_pivot[[c for c in df_pivot.columns if c!='org']].agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] == 'contribution_source':  # contributions by source
        q = '''select 
                    t.org as org,
                    t.source as source,
                    sum(t.amount) as volume,
                    count(distinct(t.id)) as count
                from production.transactions as t'''
        group_on = 't.org, t.source'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)
        df_pivot = df.pivot(index='org', columns='source', values='volume').reset_index().fillna(0)

        type_cols = [c for c in df_pivot.columns if c!='org']
        for c in df_pivot.columns:
            df_pivot['{}_percentage'.format(c)] = df_pivot[c] / df_pivot[type_cols].sum(axis=1)

        data = {
            'sample_size': len(df_pivot),
            'data': df_pivot[[c for c in df_pivot.columns if c!='org']].agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] in ['trans_count', 'trans_average', 'trans_sum']:
        # contribution count, average contribution amount, amount raised
        q = '''select 
                    t.org as org,
                    median(t.amount) as trans_median,
                    count(t.id) as trans_count,
                    sum(t.amount) as trans_volume
                from production.transactions as t'''
        group_on = 't.org'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)
        df['trans_mean'] = df['trans_volume'] / df['trans_count']

        data = {
            'sample_size': len(df_pivot),
            'data': df_pivot[[c for c in df_pivot.columns if c!='org']].agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] == 'oentime_v_recurring':  # contributions by type (one time v recurring)
        q = '''select
                    t.org as org,
                    sum(case when t.recurring=0 the 1 else 0 end) as onetime_count,
                    sum(case when t.recurring=0 the amount else 0 end) as onetime_volume,
                    sum(case when t.recurring_origin!=0 the 1 else 0 end) as recurring_count,
                    sum(case when t.recurring_origin!=0 the amount else 0 end) as recurring_volume
                from production.transactions as t'''
        group_on = 't.org'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)

        df['total_count'] = df['onetime_count'] + df['recurring_count']
        df['total_volume'] = df['onetime_volume'] + df['recurring_volume']
        df['onetime_count_percentage'] = df['onetime_count'] / df['total_count']
        df['recurring_count_percentage'] = df['recurring_count'] / df['total_count']
        df['onetime_volume_percentage'] = df['onetime_volume'] / df['total_count']
        df['recurring_volume_percentage'] = df['recurring_volume'] / df['total_count']
        df.drop(['total_count', 'total_volume'], axis=1, inplace=True)

        data = {
            'sample_size': len(df),
            'data': df[[c for c in df.columns if c!='org']].agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] in ['donation_count', 'registration_count']:
        # donation count, registration count
        q = '''select
                    t.org as org,
                    sum(t.donation_count) as donations_count,
                    sum(t.donation_amt) as donations_volume,
                    sum(t.registration_count) as registrations_count,
                    sum(t.registration_amt) as registrations_volume
                from production.transactions as t'''
        group_on = 't.org'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)

        df['total_count'] = df['donations_count'] + df['registrations_count']
        df['total_volume'] = df['donations_volume'] + df['registrations_volume']
        df['donations_count_percentage'] = df['donations_count'] / df['total_count']
        df['registrations_count_percentage'] = df['registrations_count'] / df['total_count']
        df['donations_volume_percentage'] = df['donations_volume'] / df['total_count']
        df['registrations_volume_percentage'] = df['registrations_volume'] / df['total_count']
        df.drop(['total_count', 'total_volume'], axis=1, inplace=True)

        data = {
            'sample_size': len(df),
            'data': df[[c for c in df.columns if c!='org']].agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] == 'giftassist':
        # total gift assist, gift assist success rate
        q = '''select
                    t.org as org,
                    count(distinct(t.id)) as trans_count,
                    sum(t.gift_assist_count) as giftassist_count,
                    sum(t.amount) as volume,
                    sum(t.gift_assist_amt) as giftassist_volume
                from production.transactions as t'''
        group_on = 't.org'

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)

        df['success_rate'] = df['giftassist_count'] / df['trans_count']
        df.drop(['org', 'trans_count'], axis=1, inplace=True)

        data = {
            'sample_size': len(df_pivot),
            'data': df_pivot.agg(['mean', 'median']).to_dict()
        }
    elif _input['insight'] == 'donor_liftetime_value':
        # public.donors_orgs, volume
        q = '''select
                    count(distinct(do.org)) as sample_size,
                    avg(do.volume) as average_lifetime_value
                from public.donors_orgs as do'''
        joins.append('production.transactions as t on t.org=do.org')
        
        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)

        e = df.iloc[0]
        data = {
            'sample_size': e['sample_size'],
            'data': e['average_lifetime_value']
        }
    elif _input['insight'] in ['churn', 'retention', 'new_donors']:
        # public.org_retention
        # org, retention, churn, new_donors
        q = '''select
                    count(distinct(or.org)) as sample_size,
                    avg(or.retention) as retention,
                    avg(or.churn) as churn,
                    avg(or.new_donors) as new_donors
                from public.org_retention as or'''
        joins.append('production.transactions as t on t.org=do.org')

        df = build_and_execute_query(q, where_clause=where_clause, parameters=params, joins=joins, group_on=group_on)

        e = df.iloc[0]
        data = {
            'sample_size': e['sample_size'],
            'data': {
                'retention': e['retention'],
                'churn': e['churn'],
                'new_donors': e['new_donors']
            }
        }
    
    return data['sample_size'], data['data']

## test

In [161]:
_input = {
    'filters': {'date_min': '2019-01-01',
        'date_max': '2020-01-01'},
    'insight': 'payment_method'
}

sample, data = get_benchmarks_data(_input)

select 
                    t.org as org,
                    t.payment_type as payment_type,
                    sum(t.amount) as volume,
                    count(distinct(t.id)) as count
                from production.transactions as t where t.date>='{}' and t.date<='{}' group by t.org, t.payment_type
['2019-01-01', '2020-01-01']


In [169]:
rsp = {
    'success': '1',
    'message': {
        'sample_size': sample,
        'data': data
    }
}
json.dumps(rsp)

'{"success": "1", "message": {"sample_size": 3112, "data": {"AM": {"mean": 11598.901320694076, "median": 526.2}, "BC": {"mean": 26.773778920308484, "median": 0.0}, "DC": {"mean": 34568.4623586118, "median": 100.0}, "FB": {"mean": 46.25233933161953, "median": 0.0}, "MC": {"mean": 221744.07573586173, "median": 2129.245}, "PP": {"mean": 449.1160443444731, "median": 0.0}, "VS": {"mean": 786237.810009639, "median": 6743.405000000001}, "ca": {"mean": 7821.504771850901, "median": 0.0}, "ch": {"mean": 5172.2509125964, "median": 0.0}, "ck": {"mean": 3821.349402313624, "median": 0.0}, "mg": {"mean": 248.8891131105399, "median": 0.0}, "no": {"mean": 170180.77644280207, "median": 0.0}, "AM_percentage": {"mean": 0.11531414309000083, "median": 0.05050636310869697}, "BC_percentage": {"mean": 0.004007708000944402, "median": 0.0}, "DC_percentage": {"mean": 0.023254774308018204, "median": 0.004438997179469049}, "FB_percentage": {"mean": 0.00030069629699549757, "median": 0.0}, "MC_percentage": {"mean": 0

In [167]:
sample, data

(3112,
 {'AM': {'mean': 11598.901320694076, 'median': 526.2},
  'BC': {'mean': 26.773778920308484, 'median': 0.0},
  'DC': {'mean': 34568.4623586118, 'median': 100.0},
  'FB': {'mean': 46.25233933161953, 'median': 0.0},
  'MC': {'mean': 221744.07573586173, 'median': 2129.245},
  'PP': {'mean': 449.1160443444731, 'median': 0.0},
  'VS': {'mean': 786237.810009639, 'median': 6743.405000000001},
  'ca': {'mean': 7821.504771850901, 'median': 0.0},
  'ch': {'mean': 5172.2509125964, 'median': 0.0},
  'ck': {'mean': 3821.349402313624, 'median': 0.0},
  'mg': {'mean': 248.8891131105399, 'median': 0.0},
  'no': {'mean': 170180.77644280207, 'median': 0.0},
  'AM_percentage': {'mean': 0.11531414309000083,
   'median': 0.05050636310869697},
  'BC_percentage': {'mean': 0.004007708000944402, 'median': 0.0},
  'DC_percentage': {'mean': 0.023254774308018204,
   'median': 0.004438997179469049},
  'FB_percentage': {'mean': 0.00030069629699549757, 'median': 0.0},
  'MC_percentage': {'mean': 0.188587808713

In [90]:
df_pivot = data['df']
df_pivot[[c for c in df_pivot.columns if c!='org']].agg(['mean', 'median'])

type,auction,barnstorm,p2p,qgiv,auction_percentage,barnstorm_percentage,p2p_percentage,qgiv_percentage
mean,23260.865329,6893.990827,543130.620364,3.783630e+06,0.078124,0.013273,0.10924,0.799363
median,0.000000,0.000000,0.000000,1.375483e+04,0.000000,0.000000,0.00000,1.000000
